In [136]:
include("utils/preprocessing.jl")
include("utils/model_evaluation.jl")
include("utils/data_loader.jl")


Main.DataLoader

In [137]:
using .DataLoader

data = DataLoader.load_data("dataset/star_classification.csv");
println(data[1,:])
println(size(data,2))


Any["obj_ID", "alpha", "delta", "u", "g", "r", "i", "z", "run_ID", "rerun_ID", "cam_col", "field_ID", "spec_obj_ID", "class", "redshift", "plate", "MJD", "fiber_ID"]
18


In [138]:
using .Preprocessing

inputs, targets = Preprocessing.preprocess_data(data)
println(inputs[1,:])
println(targets[1,:])

Float32[0.9987257, 0.9988122, 0.3534751, 0.3027442, 0.99872005]
Bool[1, 0, 0]
